## XModalix Port First Milestone: ImageVAE + Loader


### Outcome
- Have a image loader and image VAE
- Train this in a notebook
    - With c. elegans and MNIST images


### Checks
- Check loss curves
- Check image recons
- 


### Steps
- Prepare datasets
    - Maybe already done in 00 notebook
- Prepare config
- Write ImageDataset Class
- Write / port ImageVAE

#### IMPORTS

In [1]:
import os
from autoencodix.utils.default_config import DataConfig, DataInfo, DefaultConfig
import autoencodix as acx

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
print(notebook_dir)
os.chdir(notebook_dir)
os.chdir("..")
print(os.getcwd())


/Users/maximilianjoas/development/autoencodix_package/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/maximilianjoas/development/autoencodix_package/notebooks
/Users/maximilianjoas/development/autoencodix_package


#### GLOBALS


In [2]:
IMGROOT = os.path.join("data/images/ALY-2_SYS721/")
IMGMAPPING = os.path.join("data/ALY-2_SYS721_mappings.txt")
NUMFILE = os.path.join("data/AM3_NO2_raw_cell.tsv")

img_config = DefaultConfig(
    checkpoint_interval=10,
    class_param="early",
    epochs=1,
    pretrain_epochs=0,
    batch_size=64,
    data_config=DataConfig(
        data_info={
            "IMG": DataInfo(
                file_path=IMGROOT,
                data_type="IMG",
                translate_direction="to",
                pretrain_epochs=1
            ),
            "RNA": DataInfo(
                file_path=NUMFILE,
                data_type="NUMERIC",
                translate_direction="from",
            ),
            "RNA2": DataInfo(
                file_path=NUMFILE,
                data_type="NUMERIC",
            ),
            "ANNO": DataInfo(
                file_path=IMGMAPPING,
                data_type="ANNOTATION",
            ),
        },
    ),
)

In [3]:
xmodalix = acx.XModalix(config=img_config)
ds = xmodalix.preprocess()

preprocessing
Checking data type: IMG
Found image type in config
current data info: file_path='data/images/ALY-2_SYS721/' data_type='IMG' scaling='STANDARD' filtering='VAR' sep=None extra_anno_file=None is_single_cell=False min_cells=0.05 min_genes=0.02 selected_layers=['X'] is_X=False normalize_counts=True log_transform=True k_filter=20 img_root=None img_width_resize=64 img_height_resize=64 translate_direction='to' pretrain_epochs=1
Checking data type: RNA
Checking data type: RNA2
Checking data type: ANNO
Given image size is possible, rescaling images to: 64x64
reading annotation file: data/ALY-2_SYS721_mappings.txt
 n_samples: {'multi_sc': {'multi_sc': 0}, 'multi_bulk': {'RNA': 260, 'RNA2': 260}, 'annotation': {'paired': 260}, 'img': {'IMG': 260}, 'from_modality': {}, 'to_modality': {}, 'paired_count': {'paired_count': 260}}
Converting 182 images to torch.float32 tensors...
Converting 52 images to torch.float32 tensors...
Converting 26 images to torch.float32 tensors...
key: train, t

/Users/maximilianjoas/development/autoencodix_package/src/autoencodix/utils/_imgreader.py:251: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  annotation = pd.read_csv(anno_file, sep=sep)


In [4]:
import torch

torch.autograd.set_detect_anomaly(True)
xmodalix.fit()

Check if we need to pretrain: multi_bulk.RNA
pretrain epochs : 0
No pretraining for multi_bulk.RNA
Check if we need to pretrain: multi_bulk.RNA2
pretrain epochs : 0
No pretraining for multi_bulk.RNA2
Check if we need to pretrain: img.IMG
pretrain epochs : 1
Starting Pretraining for: img.IMG with <class 'autoencodix.trainers._general_trainer.GeneralTrainer'>
Epoch 1 - Train Loss: 3.7703
Sub-losses: recon_loss: 3.7703, var_loss: 10.5543, anneal_factor: 0.0000, effective_beta_factor: 0.0000
Epoch 1 - Valid Loss: 1.2484
Sub-losses: recon_loss: 1.2484, var_loss: 0.0106, anneal_factor: 0.0000, effective_beta_factor: 0.0000
--- Epoch 1/1 ---
split: train, n_samples: 1315
Epoch 1/1 - Train Loss: 0.2161
no common ids
no common ids
split: valid, n_samples: 128
Epoch 1/1 - Valid Loss: 0.3579
Storing checkpoint for epoch 0...


In [5]:
len(xmodalix._trainer._trainloader)

3

In [6]:
xmodalix.save("xmodalix.pkl")

Pipeline object saved successfully.
Preprocessor saved successfully.
Model state saved successfully.


In [7]:
xmodalix = acx.XModalix.load("xmodalix.pkl")

Attempting to load a pipeline from xmodalix.pkl...
Pipeline object loaded successfully. Actual type: XModalix
Preprocessor loaded successfully.
Model state loaded successfully.


In [8]:
xmodalix.predict()

Prediction complete.
Processing latent space results into a single AnnData object...
Identified source modality for latent space: 'multi_bulk.RNA'
  - Added 10 source feature IDs to .uns
Finished processing latent results.


Result Object Public Attributes:
------------------------------
latentspaces: TrainingDynamics object
sample_ids: TrainingDynamics object
reconstructions: TrainingDynamics object
mus: TrainingDynamics object
sigmas: TrainingDynamics object
losses: TrainingDynamics object
sub_losses: LossRegistry(_losses={'total_loss': TrainingDynamics(), 'adver_loss': TrainingDynamics(), 'aggregated_sub_losses': TrainingDynamics(), 'paired_loss': TrainingDynamics(), 'class_loss': TrainingDynamics(), 'multi_bulk.RNA.recon_loss': TrainingDynamics(), 'multi_bulk.RNA.var_loss': TrainingDynamics(), 'multi_bulk.RNA.anneal_factor': TrainingDynamics(), 'multi_bulk.RNA.effective_beta_factor': TrainingDynamics(), 'multi_bulk.RNA.loss': TrainingDynamics(), 'multi_bulk.RNA2.recon_loss': TrainingDynamics(), 'multi_bulk.RNA2.var_loss': TrainingDynamics(), 'multi_bulk.RNA2.anneal_factor': TrainingDynamics(), 'multi_bulk.RNA2.effective_beta_factor': TrainingDynamics(), 'multi_bulk.RNA2.loss': TrainingDynamics(), 'img.

In [9]:
r = xmodalix.result


In [10]:
r.reconstructions.get(epoch=-1, split="test").keys()

dict_keys(['multi_bulk.RNA', 'multi_bulk.RNA2', 'img.IMG', 'translation', 'reference_img.IMG_to_img.IMG'])

In [11]:
r.latentspaces.get(epoch=-1, split="test")

{'multi_bulk.RNA': array([[-0.97644997,  0.2549662 ,  0.4736903 , ...,  1.3632005 ,
          1.8084543 , -0.47529218],
        [ 1.1430972 , -0.28743494,  0.01411211, ...,  0.9934586 ,
          0.5173479 ,  0.81855476],
        [-1.1202929 ,  0.43084484,  0.9610365 , ...,  0.19464494,
          0.8457572 ,  2.289999  ],
        ...,
        [ 1.8768251 ,  0.13153765, -0.82049173, ..., -0.17096105,
          0.05732483,  1.200028  ],
        [-0.8045398 , -2.945406  , -0.47816378, ...,  0.05711357,
          2.2121663 , -0.57363564],
        [-0.09666778,  0.7211533 ,  0.9420528 , ..., -1.2312989 ,
         -0.6208355 , -1.585803  ]], dtype=float32),
 'multi_bulk.RNA2': array([[ 1.43323   ,  1.5679679 ,  0.03803138, ...,  0.8165217 ,
          0.06890178,  0.0128539 ],
        [-0.6912983 , -1.7848468 , -2.2379472 , ...,  0.7768518 ,
          1.3857259 ,  0.38689446],
        [ 0.46948797, -0.45967808, -0.74696904, ..., -0.0181518 ,
         -0.3499291 , -0.2469027 ],
        ...,
  

In [28]:
r = xmodalix.result
r.adata_latent.obs_names

Index(['T_209', 'T_72', 'T_110', 'T_11', 'T_30', 'T_174', 'T_127', 'T_79',
       'T_97', 'T_180', 'T_93', 'T_8', 'T_257', 'T_157', 'T_111', 'T_14',
       'T_186', 'T_106', 'T_215', 'T_138', 'T_1', 'T_248', 'T_252', 'T_141',
       'T_50', 'T_260', 'T_53', 'T_154', 'T_256', 'T_229', 'T_151', 'T_253',
       'T_124', 'T_115', 'T_126', 'T_160', 'T_206', 'T_209', 'T_72', 'T_110',
       'T_80', 'T_11', 'T_145', 'T_30', 'T_99', 'T_79', 'T_97', 'T_157',
       'T_111', 'T_233', 'T_14', 'T_254', 'T_187', 'T_186', 'T_1', 'T_248',
       'T_252', 'T_144', 'T_53', 'T_154', 'T_225', 'T_229', 'T_253', 'T_151',
       'T_124', 'T_249', 'T_88', 'T_115', 'T_160', 'T_42', 'T_206', 'T_84',
       'T_239', 'T_11', 'T_99', 'T_146', 'T_93', 'T_8', 'T_111', 'T_254',
       'T_187', 'T_106', 'T_138', 'T_50', 'T_144', 'T_141', 'T_252', 'T_260',
       'T_256', 'T_122', 'T_124', 'T_249', 'T_88', 'T_229', 'T_126', 'T_115',
       'T_42', 'T_160', 'T_127', 'T_239'],
      dtype='object')

In [37]:
final_rec = r.final_reconstruction.datasets["img.IMG"]
inputimg = xmodalix.result.datasets.test.datasets["img.IMG"]

In [38]:
final_rec.sample_ids

['T_158',
 'T_132',
 'T_147',
 'T_82',
 'T_49',
 'T_242',
 'T_149',
 'T_131',
 'T_70',
 'T_40',
 'T_258',
 'T_119',
 'T_104',
 'T_43',
 'T_162',
 'T_94',
 'T_128',
 'T_201',
 'T_173',
 'T_77',
 'T_134',
 'T_194',
 'T_239',
 'T_51',
 'T_22',
 'T_130',
 'T_111',
 'T_117',
 'T_202',
 'T_124',
 'T_166',
 'T_246',
 'T_211',
 'T_87',
 'T_61',
 'T_76',
 'T_137',
 'T_39',
 'T_168',
 'T_95',
 'T_241',
 'T_96',
 'T_140',
 'T_181',
 'T_107',
 'T_18',
 'T_236',
 'T_144',
 'T_145',
 'T_116',
 'T_151',
 'T_58']

In [39]:
inputimg.sample_ids

['T_158',
 'T_132',
 'T_147',
 'T_82',
 'T_49',
 'T_242',
 'T_149',
 'T_131',
 'T_70',
 'T_40',
 'T_258',
 'T_119',
 'T_104',
 'T_43',
 'T_162',
 'T_94',
 'T_128',
 'T_201',
 'T_173',
 'T_77',
 'T_134',
 'T_194',
 'T_239',
 'T_51',
 'T_22',
 'T_130',
 'T_111',
 'T_117',
 'T_202',
 'T_124',
 'T_166',
 'T_246',
 'T_211',
 'T_87',
 'T_61',
 'T_76',
 'T_137',
 'T_39',
 'T_168',
 'T_95',
 'T_241',
 'T_96',
 'T_140',
 'T_181',
 'T_107',
 'T_18',
 'T_236',
 'T_144',
 'T_145',
 'T_116',
 'T_151',
 'T_58']

In [30]:
xmodalix.predict(from_key="IMG", to_key="IMG")

Prediction complete.
Processing latent space results into a single AnnData object...
Identified source modality for latent space: 'multi_bulk.RNA'
  - Added 10 source feature IDs to .uns
Finished processing latent results.


Result Object Public Attributes:
------------------------------
latentspaces: TrainingDynamics object
sample_ids: TrainingDynamics object
reconstructions: TrainingDynamics object
mus: TrainingDynamics object
sigmas: TrainingDynamics object
losses: TrainingDynamics object
sub_losses: LossRegistry(_losses={'total_loss': TrainingDynamics(), 'adver_loss': TrainingDynamics(), 'aggregated_sub_losses': TrainingDynamics(), 'paired_loss': TrainingDynamics(), 'class_loss': TrainingDynamics(), 'multi_bulk.RNA.recon_loss': TrainingDynamics(), 'multi_bulk.RNA.var_loss': TrainingDynamics(), 'multi_bulk.RNA.anneal_factor': TrainingDynamics(), 'multi_bulk.RNA.effective_beta_factor': TrainingDynamics(), 'multi_bulk.RNA.loss': TrainingDynamics(), 'multi_bulk.RNA2.recon_loss': TrainingDynamics(), 'multi_bulk.RNA2.var_loss': TrainingDynamics(), 'multi_bulk.RNA2.anneal_factor': TrainingDynamics(), 'multi_bulk.RNA2.effective_beta_factor': TrainingDynamics(), 'multi_bulk.RNA2.loss': TrainingDynamics(), 'img.

In [13]:
result = xmodalix.result

In [14]:
result.latentspaces.get()

{0: {'train': {'multi_bulk.RNA': array([[ 0.28324735, -0.6982587 ,  1.648307  , ...,  1.2660079 ,
            0.8274999 ,  1.2253741 ],
          [-0.15067166,  0.38263598,  0.0623107 , ...,  0.5944046 ,
            2.2380016 ,  0.9574687 ],
          [-0.42055243,  1.3140594 ,  0.23153353, ...,  0.26118624,
            1.8716244 , -0.8273534 ],
          ...,
          [ 2.3520243 , -0.62355036,  1.8383391 , ...,  1.0472054 ,
           -0.05663743, -0.438549  ],
          [-0.02761099, -1.1883264 ,  1.5511112 , ..., -0.05953817,
           -0.02882975, -0.41831848],
          [ 1.2889163 , -1.2106597 , -0.37240487, ..., -0.8445254 ,
           -0.35020912,  0.16282806]], dtype=float32),
   'multi_bulk.RNA2': array([[-0.09175314,  0.6845333 , -0.33275312, ..., -0.3842815 ,
           -2.1695404 , -0.86331785],
          [ 1.1781781 ,  0.7858614 ,  0.1660059 , ...,  1.8449749 ,
           -0.316186  ,  6.278187  ],
          [ 0.49679145, -2.4429462 , -0.43157452, ..., -0.62084544,
   

In [15]:
res1 = xmodalix.predict()
res2 = xmodalix.predict(from_key="IMG", to_key="IMG")
res = xmodalix.result # = res2

Prediction complete.
Processing latent space results into a single AnnData object...
Identified source modality for latent space: 'multi_bulk.RNA'
  - Added 10 source feature IDs to .uns
Finished processing latent results.
Prediction complete.
Processing latent space results into a single AnnData object...
Identified source modality for latent space: 'multi_bulk.RNA'
  - Added 10 source feature IDs to .uns
Finished processing latent results.


In [16]:
result.sub_results.keys()

dict_keys(['multi_bulk.RNA', 'multi_bulk.RNA2', 'img.IMG', 'pretrain.img.IMG'])

In [17]:
result.sub_losses.keys()

dict_keys(['total_loss', 'adver_loss', 'aggregated_sub_losses', 'paired_loss', 'class_loss', 'multi_bulk.RNA.recon_loss', 'multi_bulk.RNA.var_loss', 'multi_bulk.RNA.anneal_factor', 'multi_bulk.RNA.effective_beta_factor', 'multi_bulk.RNA.loss', 'multi_bulk.RNA2.recon_loss', 'multi_bulk.RNA2.var_loss', 'multi_bulk.RNA2.anneal_factor', 'multi_bulk.RNA2.effective_beta_factor', 'multi_bulk.RNA2.loss', 'img.IMG.recon_loss', 'img.IMG.var_loss', 'img.IMG.anneal_factor', 'img.IMG.effective_beta_factor', 'img.IMG.loss', 'clf_loss'])

In [18]:
result.sub_losses.get("total_loss").get()

{0: {'train': array(0.2160546), 'valid': array(0.35788469)}}

In [19]:
img = result.reconstructions.get(split="train", epoch=-1)["img.IMG"]

In [20]:
img.shape

(405, 1, 64, 64)

In [21]:
sample_img = img[0, :, :, :].squeeze()
sample_img.shape

(64, 64)

In [22]:
a = None
b = None
if not bool(a and b):
    print("at least one None")

at least one None


In [23]:
bool(a and b)

False

In [24]:
result.final_reconstruction

In [25]:
import matplotlib.pyplot as plt
img_trans = result.reconstructions.get(split="test")[0,0,:,:,:].squeeze()
plt.imshow(img_trans)

IndexError: too many indices for array: array is 1-dimensional, but 5 were indexed

In [ ]:
xmodalix._trainer._modality_dynamics.keys()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(sample_img)

## Xmodal_loss Visualization

In [ ]:
result.sub_losses.keys()
result.sub_losses.get("img.IMG.var_loss").get()


In [ ]:
import matplotlib.pyplot as plt


def plot_losses(result):
    """
    Generates and displays plots for all relevant losses from a Result object.

    Args:
        result: Your custom result object containing the sub_losses attribute.
    """
    if not hasattr(result, "sub_losses"):
        print(
            "Error: The provided result object does not have a 'sub_losses' attribute."
        )
        return

    # 1. Get all loss keys and filter out the factors
    all_loss_keys = result.sub_losses.keys()
    keys_to_plot = [key for key in all_loss_keys if not key.endswith("_factor")]

    if not keys_to_plot:
        print("No valid loss keys found to plot.")
        return

    # 2. Iterate through each loss and create a plot
    for loss_name in keys_to_plot:
        try:
            # Retrieve the nested dictionary for the current loss
            loss_data = result.sub_losses.get(loss_name).get()

            # 3. Unpack the data into lists for plotting
            epochs = sorted(loss_data.keys())
            train_losses = [loss_data[epoch]["train"] for epoch in epochs]

            # Check if validation data exists before trying to plot it
            has_valid_data = all("valid" in loss_data[epoch] for epoch in epochs)
            if has_valid_data:
                valid_losses = [loss_data[epoch]["valid"] for epoch in epochs]

            # 4. Create the plot
            plt.figure(figsize=(10, 6))
            plt.plot(epochs, train_losses, "o-", label=f"Train ({loss_name})")
            if has_valid_data:
                plt.plot(epochs, valid_losses, "x-", label=f"Validation ({loss_name})")

            plt.title(f"Training and Validation Loss: {loss_name}")
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.legend()
            plt.grid(True)
            plt.show()

        except (AttributeError, TypeError, KeyError) as e:
            print(f"Could not plot '{loss_name}'. Error retrieving data: {e}")


# This is a mock setup to demonstrate how to use the function.
print("Generating plots from mock result object...")
plot_losses(result)

In [ ]:
loss_cl = xmodalix._trainer._loss_fn
batch_dynamics = xmodalix._trainer._modality_dynamics
batch = next(iter(xmodalix._trainer._trainloader))
config = img_config
print(config.loss_reduction)

In [ ]:
# img_config.class_param = "early"
for mod_name, mod_data in batch.items():
    metadata_df = mod_data.get("metadata")


In [ ]:
  def _calc_class_loss(
        self,
        batch: Dict[str, Dict[str, Any]],
        modality_dynamics: Dict[str, Dict[str, Any]],
    ) -> torch.Tensor:
        
        device = next(iter(modality_dynamics.values()))["mp"].latentspace.device

        if not self.class_param:
            return torch.tensor(0.0, device=device)

        # Step 1: Dynamically update maps and discover new classes from the batch
        for mod_name, mod_data in batch.items():
            metadata_df = mod_data.get("metadata")
            if metadata_df is None:
                continue

            batch_map = metadata_df[self.class_param]
            self.sample_to_class_map.update(batch_map.to_dict())

            for label in batch_map.unique():
                if label not in self.class_means:
                    self.class_means[label] = torch.zeros(self.latent_dim, device=device)

        # Step 2: Calculate the loss for the current batch
        total_class_loss = torch.tensor(0.0, device=device)
        for mod_name, mod_data in batch.items():
            if mod_data.get("metadata") is None:
                continue
                
            latents = modality_dynamics[mod_name]["mp"].latentspace
            metadata_df = mod_data["metadata"]
            batch_class_labels = metadata_df[self.class_param].tolist()

            target_means_list = [self.class_means[label] for label in batch_class_labels]
            target_means_tensor = torch.stack(target_means_list).to(latents.device)
            
            distance = torch.abs(latents - target_means_tensor).mean(dim=1)
            total_class_loss += self.reduction_fn(distance)

        num_modalities_in_batch = len(batch)
        return total_class_loss / num_modalities_in_batch if num_modalities_in_batch > 0 else total_class_loss

    def update_class_means(self, epoch_dynamics: List[Dict], device):
        """
        Method to be called by the Trainer at the end of an epoch to update the
        target class mean vectors.
        """
        if not self.class_param or not epoch_dynamics:
            return

        final_latents = defaultdict(list)
        final_sample_ids = defaultdict(list)
        for batch_data in epoch_dynamics:
            for mod_name, data in batch_data["latentspaces"].items():
                final_latents[mod_name].append(data)
            for mod_name, data in batch_data["sample_ids"].items():
                final_sample_ids[mod_name].append(data)

        all_latents_df_list = []
        for mod_name in final_latents.keys():
            mod_latents = np.concatenate(final_latents[mod_name])
            mod_ids = np.concatenate(final_sample_ids[mod_name])
            all_latents_df_list.append(pd.DataFrame(mod_latents, index=mod_ids))

        if not all_latents_df_list:
            return

        all_latents_df = pd.concat(all_latents_df_list)
        all_latents_df["class_label"] = all_latents_df.index.map(self.sample_to_class_map)

        new_means_df = all_latents_df.groupby("class_label").mean()

        for label, mean_values in new_means_df.iterrows():
            self.class_means[label] = torch.tensor(
                mean_values.values, dtype=torch.float32, device=device
            )

In [ ]:
metadata_df

```python
sample_ids = {}

for k,v in batch.items():

    print(v.keys())

sample_ids



output

sample_ids = {}

for k,v in batch.items():

    print(v.keys())

sample_ids



with metadata lookin like

batch["multi_bulk.RNA"]["metadata"]```

In [ ]:
latenspaces = {}
for k, v in batch_dynamics.items():
    latenspaces[k] = v["mp"].latentspace.detach()
    # latenspaces[k] = v["la"]
latenspaces

In [ ]:
batch["multi_bulk.RNA"]["metadata"].head()

In [ ]:
sample_ids = {}
for k, v in batch.items():
    print(v.keys())
sample_ids

In [ ]:
from typing import Dict, Any

import torch
import itertools
from collections import defaultdict


def calculate_paired_loss(
    latentspaces: dict[str, torch.Tensor],
    sample_ids: dict[str, list],
    reduction: str = "mean",
    device: str = "mps",
) -> torch.Tensor:
    """
    Calculates the paired distance loss across all pairs of modalities in a batch.

    Args:
        latentspaces: A dictionary mapping modality names to their latent space tensors.
                      e.g., {'RNA': tensor_rna, 'ATAC': tensor_atac}
        sample_ids: A dictionary mapping modality names to their list of sample IDs.
        reduction: The type of reduction to apply ('mean' or 'sum').

    Returns:
        A single scalar tensor representing the total paired loss.
    """

    total_paired_loss = torch.tensor(0.0, device=device)
    modality_names = list(latentspaces.keys())

    # 1. Iterate through all unique pairs of modalities
    for mod_a, mod_b in itertools.combinations(modality_names, 2):
        ids_a = sample_ids[mod_a]
        ids_b = sample_ids[mod_b]

        # 2. Find the intersection of sample IDs
        common_ids = set(ids_a) & set(ids_b)

        if not common_ids:
            print("no common ids")
            continue  # No overlapping samples for this pair, skip

        # 3. Create a mapping from sample ID to index for efficient lookup
        id_to_idx_a = {sample_id: i for i, sample_id in enumerate(ids_a)}
        id_to_idx_b = {sample_id: i for i, sample_id in enumerate(ids_b)}

        # Get the corresponding indices for the common samples
        indices_a = [id_to_idx_a[common_id] for common_id in common_ids]
        indices_b = [id_to_idx_b[common_id] for common_id in common_ids]

        # 4. Select the latent vectors for the paired samples
        paired_latents_a = latentspaces[mod_a][indices_a]
        paired_latents_b = latentspaces[mod_b][indices_b]

        # 5. Calculate the distance between the aligned latent vectors
        # L1 distance, averaged over latent dimensions and then over samples
        distance = torch.abs(paired_latents_a - paired_latents_b).mean(dim=1)

        if reduction == "mean":
            pair_loss = distance.mean()
        elif reduction == "sum":
            pair_loss = distance.sum()
        else:
            raise ValueError(f"Reduction must be 'mean' or 'sum', not {reduction}")

        # 6. Aggregate the loss
        total_paired_loss += pair_loss

    return total_paired_loss

In [ ]:
calculate_paired_loss(latentspaces=latenspaces, sample_ids=sample_ids)

## Imagix
If we want to use a standard image vae, we can use the Imagix pipeline. Note that only one image dataset is allowed here
**NOTE**:
set the datacase in config to DataCase.IMG_TO_IMG

In [ ]:
import autoencodix as acx
from autoencodix.utils.default_config import (
    DefaultConfig,
    DataConfig,
    DataCase,
    DataInfo,
)
import os

IMGROOT = os.path.join("data/images/ALY-2_SYS721/")
IMGMAPPING = os.path.join("data/ALY-2_SYS721_mappings.txt")
NUMFILE = os.path.join("data/AM3_NO2_raw_cell.tsv")

img_config2 = DefaultConfig(
    data_case=DataCase.IMG_TO_IMG,
    data_config=DataConfig(
        data_info={
            "IMG": DataInfo(
                file_path=IMGROOT,
                data_type="IMG",
            ),
            "ANNO": DataInfo(
                file_path=IMGMAPPING,
                data_type="ANNOTATION",
            ),
        },
    ),
)

imagix = acx.Imagix(config=img_config2)
imagix.preprocess()
imagix.fit()

In [ ]:
imagix.predict()

In [ ]:
r = imagix.result
r.final_reconstruction
sample_img = r.final_reconstruction.data[0, :, :, :]
sample_img = sample_img.squeeze()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(sample_img.numpy())

### Old, maybe reuse code for getting stats, but seemd to work

In [ ]:
# Simple statistics tracking - just build the dict in your training loop

# Initialize stats dict once before training
stats = {}
for modality in trainset.datasets.keys():
    stats[modality] = {}
    for sample_id in trainset.datasets[modality].sample_ids:
        stats[modality][sample_id] = 0
print(stats)
# Training loop
epochs = 100

# Add this to your training loop
for epoch in range(100):  # Just test first 5 epochs
    batch_count = 0
    for batch_idx, batch in enumerate(dataloader):
        batch_count += 1

    print(f"Epoch {epoch}: {batch_count} batches processed")

    # Also check sampler length
    print(f"Sampler length: {len(sampler)}")
    print(f"Dataset length: {len(trainset)}")
    print(f"Paired samples: {len(trainset.paired_sample_ids)}")
    print(f"Unpaired samples: {len(trainset.unpaired_sample_ids)}")
    # # Reset stats for new epoch
    # for modality in stats:
    #     for sample_id in stats[modality]:
    #         stats[modality][sample_id] = 0

    for batch_idx, batch in enumerate(dataloader):
        # Update stats with current batch
        for modality, data in batch.items():
            if "sample_ids" in data:
                for sample_id in data["sample_ids"]:
                    if sample_id in stats[modality]:
                        stats[modality][sample_id] += 1

        # Your training code here
        # for modality, data in batch.items():
        #     if data["data"] is not None:
        #         outputs = models[modality](data["data"])

    # Print simple summary at end of epoch
    print("\nSampling Stats:")
    for modality in stats:
        counts = list(stats[modality].values())
        seen = sum(1 for c in counts if c > 0)
        total = len(counts)
        avg = sum(counts) / len(counts) if counts else 0
        print(f"  {modality}: {seen}/{total} samples seen, avg: {avg:.2f}")

    # Optional: Print unseen samples
    print("\nUnseen samples:")
    for modality in stats:
        unseen = [sid for sid, count in stats[modality].items() if count == 0]
        if unseen:
            print(
                f"  {modality}: {len(unseen)} unseen - {unseen[:5]}{'...' if len(unseen) > 5 else ''}"
            )

# Access raw stats anytime:
# stats = {
#     "img.IMG": {"T_98": 3, "T_138": 2, "T_183": 1, ...},
#     "multi_bulk.RNA": {"T_98": 4, "T_138": 3, "T_173": 2, ...},
#     ...
# }